# Chapter 9: Learning and Adaptation

Key Takeaways:
- **Evolutionary Coding** uses LLMs to automatically discover and optimize algorithms through iterative code generation, evaluation, and selection.
- **OpenEvolve** is an open-source implementation of evolutionary coding that can evolve entire code files across multiple programming languages.
- **Multi-objective Optimization** allows simultaneous optimization of multiple metrics like correctness, performance, and code quality.

### Heuristic: *Evolution over manual optimization.*

## Setup and Initialization

In [1]:
import os
import nest_asyncio
from dotenv import load_dotenv

# Allow nested event loops (required for OpenEvolve in Jupyter)
nest_asyncio.apply()

load_dotenv()

# --- Configuration ---
PROJECT_ROOT = os.path.dirname(os.getcwd())
SCRIPTS_DIR = os.path.join(PROJECT_ROOT, "scripts")

print(f"✅ Configuration Loaded:")
print(f"   Project Root: {PROJECT_ROOT}")
print(f"   Scripts Directory: {SCRIPTS_DIR}")

✅ Configuration Loaded:
   Project Root: /Users/jorgemartinez/Documents/projects/agentic-design-patterns
   Scripts Directory: /Users/jorgemartinez/Documents/projects/agentic-design-patterns/scripts


## OpenEvolve Overview

OpenEvolve is an evolutionary coding agent that leverages Large Language Models (LLMs) to automatically optimize and discover algorithms. It implements the key concepts from Google DeepMind's AlphaEvolve system.

### Key Components

| Component | Description |
|-----------|-------------|
| **LLM Ensemble** | Uses multiple language models to generate diverse code modifications |
| **Prompt Sampler** | Creates context-rich prompts incorporating past programs and scores |
| **Evaluator Pool** | Tests generated programs and assigns scores based on defined metrics |
| **Program Database** | Stores evolved programs using MAP-Elites algorithm for diversity |

## The Evolution Pipeline

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Initial Code   │────▶│   LLM Mutates   │────▶│    Evaluate     │
│   (Program)     │     │   (Generate)    │     │   (Score)       │
└─────────────────┘     └─────────────────┘     └────────┬────────┘
                                                         │
                                                         ▼
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Best Program   │◀────│    Selection    │◀────│ Program Database│
│   (Output)      │     │   (Fitness)     │     │   (Archive)     │
└─────────────────┘     └─────────────────┘     └─────────────────┘
```

## Required Files

OpenEvolve requires three key files to operate:

1. **Initial Program** (`initial_program.py`) - The starting code to evolve
2. **Evaluator** (`evaluator.py`) - Defines how to score evolved programs
3. **Config** (`config.yaml`) - Evolution parameters and LLM settings

### 1. Initial Program

This is the starting point for evolution. Mark sections with `EVOLVE-BLOCK-START` and `EVOLVE-BLOCK-END` comments to indicate which parts can be modified.

In [2]:
# View the initial program
initial_program_path = os.path.join(SCRIPTS_DIR, "initial_program.py")

with open(initial_program_path, 'r') as f:
    print(f.read())

"""
Initial Program for OpenEvolve Evolution
This is a simple sorting function that OpenEvolve will attempt to optimize.
The algorithm starts as a basic bubble sort and can be evolved into more
efficient implementations.

This file contains the code that will be evolved by OpenEvolve.
Mark sections with # @evolve comments to indicate which parts can be modified.
"""


# EVOLVE-BLOCK-START
def sort_numbers(arr: list[int]) -> list[int]:
    """
    Sort a list of numbers in ascending order.
    
    This is the target function for evolution. OpenEvolve will attempt
    to improve its performance while maintaining correctness.
    
    Args:
        arr: A list of integers to sort
        
    Returns:
        A new list with elements sorted in ascending order
    """
    # Basic bubble sort - initial implementation to be evolved
    result = arr.copy()
    n = len(result)
    
    for i in range(n):
        for j in range(0, n - i - 1):
            if result[j] > result[j + 1]:
         

### 2. Evaluator

The evaluator defines the fitness function that scores each evolved program. It should return a dictionary of metrics.

In [3]:
# View the evaluator
evaluator_path = os.path.join(SCRIPTS_DIR, "evaluator.py")

with open(evaluator_path, 'r') as f:
    print(f.read())

"""
Evaluator for OpenEvolve Evolution
This module defines the evaluation function that scores evolved programs.
OpenEvolve uses this to determine which program variants are better.

The evaluator returns a dictionary of metrics that OpenEvolve uses for
multi-objective optimization.

IMPORTANT: OpenEvolve passes a FILE PATH to the evaluate function, not a module.
"""

import time
import random
import importlib.util
import sys
from pathlib import Path
from typing import Any


def evaluate(program_path: str) -> dict[str, float]:
    """
    Evaluate an evolved program and return performance metrics.
    
    This function is called by OpenEvolve to score each candidate program.
    It should return a dictionary of metric names to scores, where higher
    scores are better.
    
    Args:
        program_path: Path to the evolved program file
        
    Returns:
        Dictionary of metric names to scores (higher is better)
    """
    metrics = {}
    
    # Load the program module fr

### 3. Configuration

The config file controls evolution parameters, LLM settings, and evaluation options.

In [4]:
# View the configuration
config_path = os.path.join(SCRIPTS_DIR, "config.yaml")

with open(config_path, 'r') as f:
    print(f.read())

# OpenEvolve Configuration
# =========================
# This configuration file controls the evolution process.

# ========================
# Top-Level Settings
# ========================

# Number of iterations to run
max_iterations: 100

# Save checkpoint every N iterations
checkpoint_interval: 10

# Log level: DEBUG, INFO, WARNING, ERROR
log_level: "INFO"

# Log directory (optional)
log_dir: "../scripts/evolution_output/evolution_logs"

# Random seed for reproducibility
random_seed: 42

# ========================
# LLM Configuration
# ========================
llm:
  # Primary model for code generation
  primary_model: "gpt-4o-mini"
  # Secondary model for diversity (optional)
  secondary_model: "gpt-4o-mini"
  # Temperature for generation (higher = more creative)
  temperature: 0.7
  # Maximum tokens per generation
  max_tokens: 2048

# ========================
# Evaluator Configuration
# ========================
evaluator:
  # Disable cascade evaluation (our evaluator uses direct 

## Running Evolution

OpenEvolve provides a simple `run_evolution` API that accepts file paths directly:

In [5]:
from openevolve import run_evolution

# Define file paths
initial_program_path = os.path.join(SCRIPTS_DIR, "initial_program.py")
evaluator_path = os.path.join(SCRIPTS_DIR, "evaluator.py")
config_path = os.path.join(SCRIPTS_DIR, "config.yaml")

print("📁 OpenEvolve Files:")
print(f"   Initial Program: {initial_program_path}")
print(f"   Evaluator: {evaluator_path}")
print(f"   Config: {config_path}")
print("\n✅ Ready to run evolution!")

📁 OpenEvolve Files:
   Initial Program: /Users/jorgemartinez/Documents/projects/agentic-design-patterns/scripts/initial_program.py
   Evaluator: /Users/jorgemartinez/Documents/projects/agentic-design-patterns/scripts/evaluator.py
   Config: /Users/jorgemartinez/Documents/projects/agentic-design-patterns/scripts/config.yaml

✅ Ready to run evolution!


In [ ]:
# Run the evolution process
# Note: This requires an LLM API key (e.g., OPENAI_API_KEY or GOOGLE_API_KEY)
# In a real scenario, you would run many more iterations

result = run_evolution(
    initial_program=initial_program_path,
    evaluator=evaluator_path,
    config=config_path,
    iterations=2,  # Small number for demo; use 100+ in production
    output_dir="../scripts/openevolve_output"
)

print(f"\n🏆 Evolution Complete!")
print(f"Best program metrics:")
for name, value in result.metrics.items():
    print(f"  {name}: {value:.4f}")

2026-01-10 13:23:15,606 - INFO - Logging to ../scripts/evolution_output/evolution_logs/openevolve_20260110_132315.log
2026-01-10 13:23:15,624 - INFO - Set random seed to 42 for reproducibility
2026-01-10 13:23:15,658 - INFO - Initialized OpenAI LLM with model: gpt-4o-mini
2026-01-10 13:23:15,664 - INFO - Initialized LLM ensemble with models: gpt-4o-mini (weight: 0.83), gpt-4o-mini (weight: 0.17)
2026-01-10 13:23:15,675 - INFO - Initialized LLM ensemble with models: gpt-4o-mini (weight: 0.83), gpt-4o-mini (weight: 0.17)
2026-01-10 13:23:15,678 - INFO - Initialized prompt sampler
2026-01-10 13:23:15,679 - INFO - Set custom templates: system=evaluator_system_message, user=None
2026-01-10 13:23:15,679 - INFO - Initialized program database with 0 programs
2026-01-10 13:23:15,681 - INFO - Successfully loaded evaluation function from /Users/jorgemartinez/Documents/projects/agentic-design-patterns/scripts/evaluator.py
2026-01-10 13:23:15,681 - INFO - Initialized evaluator with /Users/jorgemart


🏆 Evolution Complete!
Best program metrics:
  correctness: 1.0000
  performance: 0.9996
  combined_score: 0.9999


## Analyzing Results

After evolution, you can examine the best program and its improvements:

In [7]:
# View the evolved code
print("📝 Evolved Program Code:")
print("=" * 50)
print(result.best_code)
print("=" * 50)

# Compare metrics with the original
print(f"\n📊 Performance Improvement:")
print(f"  Correctness: {result.metrics.get('correctness', 0):.2%}")
print(f"  Performance: {result.metrics.get('performance', 0):.2%}")
print(f"  Overall Fitness: {result.metrics.get('fitness', 0):.2%}")

📝 Evolved Program Code:
"""
Initial Program for OpenEvolve Evolution
This is a simple sorting function that OpenEvolve will attempt to optimize.
The algorithm starts as a basic bubble sort and can be evolved into more
efficient implementations.

This file contains the code that will be evolved by OpenEvolve.
Mark sections with # @evolve comments to indicate which parts can be modified.
"""


# EVOLVE-BLOCK-START
def sort_numbers(arr: list[int]) -> list[int]:
    """
    Sort a list of numbers in ascending order.
    
    This is the target function for evolution. OpenEvolve will attempt
    to improve its performance while maintaining correctness.
    
    Args:
        arr: A list of integers to sort
        
    Returns:
        A new list with elements sorted in ascending order
    """
    # Using Python's built-in sort for improved performance
    return sorted(arr)
# EVOLVE-BLOCK-END


if __name__ == "__main__":
    # Test the sorting function
    test_data = [64, 34, 25, 12, 22, 

## Alternative: Using the Low-Level API

For more control, you can use the `OpenEvolve` class directly with a `Config` object:

In [8]:
from openevolve import OpenEvolve
from openevolve.config import Config

# Load configuration from YAML file
config = Config.from_yaml(os.path.join(SCRIPTS_DIR, "config.yaml"))

# Initialize the OpenEvolve controller
evolve = OpenEvolve(
    initial_program_path=os.path.join(SCRIPTS_DIR, "initial_program.py"),
    evaluation_file=os.path.join(SCRIPTS_DIR, "evaluator.py"),
    config=config
)

print("✅ OpenEvolve initialized with Config object!")

2026-01-10 13:23:28,092 - INFO - Logging to ../scripts/evolution_output/evolution_logs/openevolve_20260110_132328.log
2026-01-10 13:23:28,092 - INFO - Logging to ../scripts/evolution_output/evolution_logs/openevolve_20260110_132328.log
2026-01-10 13:23:28,092 - INFO - Set random seed to 42 for reproducibility
2026-01-10 13:23:28,092 - INFO - Set random seed to 42 for reproducibility
2026-01-10 13:23:28,107 - INFO - Initialized LLM ensemble with models: gpt-4o-mini (weight: 0.83), gpt-4o-mini (weight: 0.17)
2026-01-10 13:23:28,107 - INFO - Initialized LLM ensemble with models: gpt-4o-mini (weight: 0.83), gpt-4o-mini (weight: 0.17)
2026-01-10 13:23:28,118 - INFO - Initialized LLM ensemble with models: gpt-4o-mini (weight: 0.83), gpt-4o-mini (weight: 0.17)
2026-01-10 13:23:28,118 - INFO - Initialized LLM ensemble with models: gpt-4o-mini (weight: 0.83), gpt-4o-mini (weight: 0.17)
2026-01-10 13:23:28,121 - INFO - Set custom templates: system=evaluator_system_message, user=None
2026-01-10 1

✅ OpenEvolve initialized with Config object!


## Key Configuration Options

| Parameter | Description | Typical Range |
|-----------|-------------|---------------|
| `iterations` | Number of evolution cycles | 100-10000 |
| `population_size` | Programs maintained per generation | 10-100 |
| `temperature` | LLM creativity (higher = more diverse) | 0.5-1.0 |
| `mutation_rate` | Probability of code modification | 0.1-0.5 |
| `elite_count` | Top programs preserved each generation | 1-5 |

## Best Practices

1. **Start Simple** - Begin with a basic, working implementation
2. **Clear Metrics** - Define measurable, unambiguous evaluation criteria
3. **Balanced Objectives** - Weight correctness higher than performance initially
4. **Checkpointing** - Save progress regularly for long evolution runs
5. **Diversity** - Use the MAP-Elites algorithm to avoid local optima

## Conclusion

Learning and Adaptation through evolutionary coding represents a paradigm shift in algorithm development. Instead of manually optimizing code, you can:

- **Define the goal** (evaluation metrics)
- **Provide a starting point** (initial program)
- **Let evolution discover improvements** (run OpenEvolve)

This approach is particularly powerful for complex optimization problems where human intuition may miss novel solutions.